## Porto Seguro チュートリアル *end-to-end* アンサンブル

Python 3には便利な分析ライブラリが多く含まれています。Kaggle/Pythonドッカーイメージで構築されています。 https://github.com/kaggle/docker-python

それでは、パッケージをいくつか読み込んでみましょう。

In [9]:
import numpy as np # 線形代数
import pandas as pd # データ処理： CSV ファイル I/O（例 pd.read_csv）
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

In [11]:
import xgboost as xgb
import lightgbm as lgb
import time

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 1000)

# 入力データファイルは ".../input/" ディレクトリにあります。
# runをクリック、もしくはShift+Enterを実行することで入力ディレクトリに
# 含まれているファイルを確認することができます。

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# カレントディレクトリに書いたものは全て出力として保存されます。

feature_stat.csv
sample_submission.7z
sample_submission.csv
test.7z
test.csv
train.7z
train.csv
traintest.csv
traintest_h20000.csv



### 読まれる方へ
こちらのコードはKaggleを通してスキルを身に付けたい学生向けに書かれています。LB（リーダーボード）を過学習させようと競う人も多くいる中で、今回こちらでは基本的なCV（分割交差検証）を使った方法に重点をおきます。
もちろん生徒さんの中にはチームで競う方もいらっしゃるので、カーネルを使って積極的に情報共有をしていただければと思います。是非以下にコメントも残してください。では楽しんでください！


## Porto Seguro - End-to-end アンサンブル

このチャレンジでは運転者が保険金請求を行うかどうかを予測する予測モデルを構築します。前もってデータを探求した後、いくつか便利なカテゴリカル・フィーチャー・エンコーディング（特徴量作成）を行い、モデル構築するためのシンプルなパイプラインを作成しました。

こちらのカーネルではモデリングに深入りします。まずはいくつかのモデルにout-out-fold学習・テスト予測を行うテクニックをみていきます。後にこれらをまとめてアンサンブルモデルに使います。

今回行うアンサンブルは一般的にアンサンブル学習と呼ばれ、以下のブログで上手にまとめられています。

* [Kaggle Ensemble Guide](https://mlwave.com/kaggle-ensembling-guide/)  （[Triskelion](https://www.kaggle.com/triskelion)）
* [Stacking Made Easy: An Introduction to StackNet](http://blog.kaggle.com/2017/06/15/stacking-made-easy-an-introduction-to-stacknet-by-competitions-grandmaster-marios-michailidis-kazanova/) （グランドマスター [Marios Michailidis (KazAnova)](https://www.kaggle.com/kazanova)）


では、さっそく取り掛かりましょう！

In [14]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')
sample_submission=pd.read_csv('../input/sample_submission.csv')

## 1. カテゴリカル・フィーチャー・エンコーディングとフィーチャ削減
こちらはデータ前処理のカーネルと同じですので、詳細は[こちら](https://www.kaggle.com/yifanxie/porto-seguro-tutorial-simple-e2e-pipeline)を確認してください。

### 1.1 フリークエンシー・エンコーディング

In [15]:
# 学習用データとテストデータから'cols'を取り出しフリークエンシー（出現頻度）エンコーディングを行います。
def freq_encoding(cols, train_df, test_df):
    # 新しいデータセットを以下のデータセットに保存します。
    result_train_df=pd.DataFrame()
    result_test_df=pd.DataFrame()
    
    # 各フィーチャーカラム（縦列）をループしていきます。
    for col in cols:
        
        # 学習用セットの中でフィーチャーの出現頻度をデータフレームとして読み込んでいきます。
        col_freq=col+'_freq'
        freq=train_df[col].value_counts()
        freq=pd.DataFrame(freq)
        freq.reset_index(inplace=True)
        freq.columns=[[col,col_freq]]

        # 'freq'データフレームを学習用データと融合します。
        temp_train_df=pd.merge(train_df[[col]], freq, how='left', on=col)
        temp_train_df.drop([col], axis=1, inplace=True)

        # 'freq'データフレームをテストデータと融合します。
        temp_test_df=pd.merge(test_df[[col]], freq, how='left', on=col)
        temp_test_df.drop([col], axis=1, inplace=True)

        # 学習用セットで現れなかったレベルがテストセットに現れなかった場合、頻度を０と設定します。
        temp_test_df.fillna(0, inplace=True)
        temp_test_df[col_freq]=temp_test_df[col_freq].astype(np.int32)

        if result_train_df.shape[0]==0:
            result_train_df=temp_train_df
            result_test_df=temp_test_df
        else:
            result_train_df=pd.concat([result_train_df, temp_train_df],axis=1)
            result_test_df=pd.concat([result_test_df, temp_test_df],axis=1)
    
    return result_train_df, result_test_df

フリークエンシー・エンコーディングの関数を実行してみましょう。

### 1.2 バイナリ変換

In [16]:
# カテゴリカル変数をバイナリに変換する関数を作成します。
# 学習用セットとテストセット、エンコードされるフィーチャーをとり、
# 入力フィーチャをバイナリー表示に変換されたデータセットが２つ返ってきます。
# この関数は符号化されるフィーチャが、nをフィーチャのレベル数としたとき
# 既に0からn-1の範囲で数値型に変換されていることを仮定しています。

def binary_encoding(train_df, test_df, feat):
    # 数値型変換で使用された最大値を計算。
    train_feat_max = train_df[feat].max()
    test_feat_max = test_df[feat].max()
    if train_feat_max > test_feat_max:
        feat_max = train_feat_max
    else:
        feat_max = test_feat_max
        
    # 欠損値にはfeat_max+1を使います。
    train_df.loc[train_df[feat] == -1, feat] = feat_max + 1
    test_df.loc[test_df[feat] == -1, feat] = feat_max + 1
    
    # 有り得るすべてのフィーチャの集合体を作成します。
    union_val = np.union1d(train_df[feat].unique(), test_df[feat].unique())

    # フィーチャから小数点表示で最大値を抜き出します。
    max_dec = union_val.max()
    
    # max_devをバイナリ表示するのに必要な桁数を計算します。
    max_bin_len = len("{0:b}".format(max_dec))
    index = np.arange(len(union_val))
    columns = list([feat])
    
    # フィーチャ全てのレベルを取得するのにバイナリ変換フィーチャ用のデータフレームを作成。
    bin_df = pd.DataFrame(index=index, columns=columns)
    bin_df[feat] = union_val
    
    # フィーチャの各レベルのバイナリ表示を取得。
    feat_bin = bin_df[feat].apply(lambda x: "{0:b}".format(x).zfill(max_bin_len))
    
    # バイナリ表示を異なる桁数に分割する。
    splitted = feat_bin.apply(lambda x: pd.Series(list(x)).astype(np.uint8))
    splitted.columns = [feat + '_bin_' + str(x) for x in splitted.columns]
    bin_df = bin_df.join(splitted)
    
    # バイナリ変換フィーチャ用のデータフレームを学習用セットとテストセットで結合させ、完成です！ 
    train_df = pd.merge(train_df, bin_df, how='left', on=[feat])
    test_df = pd.merge(test_df, bin_df, how='left', on=[feat])
    return train_df, test_df

バイナリ変換関数を実行してみましょう。

In [17]:
cat_cols=['ps_ind_02_cat','ps_car_04_cat', 'ps_car_09_cat',
          'ps_ind_05_cat', 'ps_car_01_cat']

train, test=binary_encoding(train, test, 'ps_ind_02_cat')
train, test=binary_encoding(train, test, 'ps_car_04_cat')
train, test=binary_encoding(train, test, 'ps_car_09_cat')
train, test=binary_encoding(train, test, 'ps_ind_05_cat')
train, test=binary_encoding(train, test, 'ps_car_01_cat')

任意でもとのカテゴリフィーチャを削除しても構いません。ここでは試しにやってみましょう。

In [18]:
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train.drop(col_to_drop, axis=1, inplace=True)  
test.drop(col_to_drop, axis=1, inplace=True)  

### 1.3 フィチャー削減
calを含むフィーチャを全て取り除いてみましょう。

In [19]:
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train.drop(col_to_drop, axis=1, inplace=True)  
test.drop(col_to_drop, axis=1, inplace=True)

上記のデータ処理のあとデータセットを簡単に見てみましょう。

In [20]:
train.head(5)

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_ind_02_cat_bin_0,ps_ind_02_cat_bin_1,ps_ind_02_cat_bin_2,ps_car_04_cat_bin_0,ps_car_04_cat_bin_1,ps_car_04_cat_bin_2,ps_car_04_cat_bin_3,ps_car_09_cat_bin_0,ps_car_09_cat_bin_1,ps_car_09_cat_bin_2,ps_ind_05_cat_bin_0,ps_ind_05_cat_bin_1,ps_ind_05_cat_bin_2,ps_car_01_cat_bin_0,ps_car_01_cat_bin_1,ps_car_01_cat_bin_2,ps_car_01_cat_bin_3
0,7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,1
2,13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1
3,16,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,1,1
4,17,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1


## 2. K-fold CV（k-分割交差検証）と Out-of-Fold (OOF) 予測
注：デモンストレーションのため、パラメータを落として実行時間を速めました。実際やる時にはパラメータの組み合わせを考えてやってみてください。

### 2.1 OOFに便利な関数の作成
まずAUCスコアからジニ係数を求める関数を書きます。

In [21]:
def auc_to_gini_norm(auc_score):
    return 2*auc_score-1

### 2.1.1 Sklearn K-fold と OOF 関数
学習データとテストデータのOOF予測を求めるk-fold関数を書きます。

In [22]:
def cross_validate_sklearn(clf, x_train, y_train , x_test, kf,scale=False, verbose=True):
    start_time=time.time()
    
    # out-of-fold学習とテスト予測のサイズを初期化。
    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))

    # k-foldオブジェクトを使い必要な分割（fold）の作成。
    for i, (train_index, test_index) in enumerate(kf.split(x_train, y_train)):
        # 学習分割（training fold）と検証分割（validation fold）を作ります。
        x_train_kf, x_val_kf = x_train.loc[train_index, :], x_train.loc[test_index, :]
        y_train_kf, y_val_kf = y_train[train_index], y_train[test_index]

        # 必要であればスケーリングを行います。（線形アルゴリズム用）
        if scale:
            scaler = StandardScaler().fit(x_train_kf.values)
            x_train_kf_values = scaler.transform(x_train_kf.values)
            x_val_kf_values = scaler.transform(x_val_kf.values)
            x_test_values = scaler.transform(x_test.values)
        else:
            x_train_kf_values = x_train_kf.values
            x_val_kf_values = x_val_kf.values
            x_test_values = x_test.values
        
        # input classifier（入力分類器）を通して予測を行います。
        clf.fit(x_train_kf_values, y_train_kf.values)
        val_pred=clf.predict_proba(x_val_kf_values)[:,1]
        train_pred[test_index] += val_pred

        y_test_preds = clf.predict_proba(x_test_values)[:,1]
        test_pred += y_test_preds

        fold_auc = roc_auc_score(y_val_kf.values, val_pred)
        fold_gini_norm = auc_to_gini_norm(fold_auc)

        if verbose:
            print('fold cv {} AUC score is {:.6f}, Gini_Norm score is {:.6f}'.format(i, fold_auc, fold_gini_norm))

    test_pred /= kf.n_splits

    cv_auc = roc_auc_score(y_train, train_pred)
    cv_gini_norm = auc_to_gini_norm(cv_auc)
    cv_score = [cv_auc, cv_gini_norm]
    if verbose:
        print('cv AUC score is {:.6f}, Gini_Norm score is {:.6f}'.format(cv_auc, cv_gini_norm))
        end_time = time.time()
        print("it takes %.3f seconds to perform cross validation" % (end_time - start_time))
    return cv_score, train_pred,test_pred

### 2.1.2 Xgboost K-分割とOOF関数
ここではネイティブインタフェースのXGBとLGBを使います。もちろんsklearnのAPIを使うのも手ですが、ネイティブインタフェースの方がいくつか利点があります。私が知っている限り、例えばXGBでヒストグラムを書くhist機能はネイティブインタフェースでしか扱われていません。

この２つのOOF関数のために確率をランクに変換させる関数も用意します。予測確率の代わりに正規化ランクを使う理由は後ほど明らかになります。

In [23]:
def probability_to_rank(prediction, scaler=1):
    pred_df=pd.DataFrame(columns=['probability'])
    pred_df['probability']=prediction
    pred_df['rank']=pred_df['probability'].rank()/len(prediction)*scaler
    return pred_df['rank'].values

次はOOF予測を計算するためにXGB用のk-fold関数を作成します。こちらはsklearnと似ていますが、分類を容易にするためにXGBインタフェースを使う必要がある上、後に確率をランクに変換させる機能を追加するためにも独自のものを作成して使います。

In [24]:
def cross_validate_xgb(params, x_train, y_train, x_test, kf, cat_cols=[], verbose=True, 
                       verbose_eval=50, num_boost_round=4000, use_rank=True):
    start_time=time.time()

    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))

    # k-foldオブジェクトを使って学習フォルドと検証フォルドそれぞれにインデックスを付けます。
    for i, (train_index, val_index) in enumerate(kf.split(x_train, y_train)): #1, 2 ,3 ,4, 5フォルド
        # 例：1,2,3,4が学習、5が検証
        x_train_kf, x_val_kf = x_train.loc[train_index, :], x_train.loc[val_index, :]
        y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
        x_test_kf=x_test.copy()

        d_train_kf = xgb.DMatrix(x_train_kf, label=y_train_kf)
        d_val_kf = xgb.DMatrix(x_val_kf, label=y_val_kf)
        d_test = xgb.DMatrix(x_test_kf)

        bst = xgb.train(params, d_train_kf, num_boost_round=num_boost_round,
                        evals=[(d_train_kf, 'train'), (d_val_kf, 'val')], verbose_eval=verbose_eval,
                        early_stopping_rounds=50)

        val_pred = bst.predict(d_val_kf, ntree_limit=bst.best_ntree_limit)
        if use_rank:
            train_pred[val_index] += probability_to_rank(val_pred)
            test_pred+=probability_to_rank(bst.predict(d_test))
        else:
            train_pred[val_index] += val_pred
            test_pred+=bst.predict(d_test)

        fold_auc = roc_auc_score(y_val_kf.values, val_pred)
        fold_gini_norm = auc_to_gini_norm(fold_auc)

        if verbose:
            print('fold cv {} AUC score is {:.6f}, Gini_Norm score is {:.6f}'.format(i, fold_auc, 
                                                                                     fold_gini_norm))

    test_pred /= kf.n_splits

    cv_auc = roc_auc_score(y_train, train_pred)
    cv_gini_norm = auc_to_gini_norm(cv_auc)
    cv_score = [cv_auc, cv_gini_norm]
    if verbose:
        print('cv AUC score is {:.6f}, Gini_Norm score is {:.6f}'.format(cv_auc, cv_gini_norm))
        end_time = time.time()
        print("it takes %.3f seconds to perform cross validation" % (end_time - start_time))

        return cv_score, train_pred,test_pred

### 2.1.3 LigthGBM K-fold と OOF 関数
LGBにも同様の関数を作成します。LightGBMインタフェースを呼ぶコード以外、XGBとほとんど同じです。

In [25]:
def cross_validate_lgb(params, x_train, y_train, x_test, kf, cat_cols=[],
                       verbose=True, verbose_eval=50, use_cat=True, use_rank=True):
    start_time = time.time()
    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))

    if len(cat_cols)==0: use_cat=False

    # k-foldオブジェクトを使って学習フォルドと検証フォルドそれぞれにインデックスを付けます。
    for i, (train_index, val_index) in enumerate(kf.split(x_train, y_train)): #1, 2 ,3 ,4, 5フォルド
        # 例：1,2,3,4が学習、5が検証
        x_train_kf, x_val_kf = x_train.loc[train_index, :], x_train.loc[val_index, :]
        y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]

        if use_cat:
            lgb_train = lgb.Dataset(x_train_kf, y_train_kf, categorical_feature=cat_cols)
            lgb_val = lgb.Dataset(x_val_kf, y_val_kf, reference=lgb_train, categorical_feature=cat_cols)
        else:
            lgb_train = lgb.Dataset(x_train_kf, y_train_kf)
            lgb_val = lgb.Dataset(x_val_kf, y_val_kf, reference=lgb_train)

        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=4000,
                        valid_sets=lgb_val,
                        early_stopping_rounds=30,
                        verbose_eval=verbose_eval)

        val_pred = gbm.predict(x_val_kf)

        if use_rank:
            train_pred[val_index] += probability_to_rank(val_pred)
            test_pred += probability_to_rank(gbm.predict(x_test))
            # test_pred += gbm.predict(x_test)　加算代入
        else:
            train_pred[val_index] += val_pred
            test_pred += gbm.predict(x_test)

        # test_pred += gbm.predict(x_test)　加算代入
        fold_auc = roc_auc_score(y_val_kf.values, val_pred)
        fold_gini_norm = auc_to_gini_norm(fold_auc)
        if verbose:
            print('fold cv {} AUC score is {:.6f}, Gini_Norm score is {:.6f}'.format(i, fold_auc, fold_gini_norm))

    test_pred /= kf.n_splits

    cv_auc = roc_auc_score(y_train, train_pred)
    cv_gini_norm = auc_to_gini_norm(cv_auc)
    cv_score = [cv_auc, cv_gini_norm]
    if verbose:
        print('cv AUC score is {:.6f}, Gini_Norm score is {:.6f}'.format(cv_auc, cv_gini_norm))
        end_time = time.time()
        print("it takes %.3f seconds to perform cross validation" % (end_time - start_time))
    return cv_score, train_pred,test_pred

## 3. レベル１OOF 予測の作成
機械学習アルゴリズムのための学習とテストデータの準備をし、StratifiedKFoldオブジェクトを作成すればレベル１OOF出力まであともう少しです。

In [26]:
drop_cols=['id','target']
y_train=train['target']
x_train=train.drop(drop_cols, axis=1)
x_test=test.drop(['id'], axis=1)

スタッキングには全てのレベルと全てのモデルに同じフォルドの分布を使うようにしましょう。技術的な理由はこのチャレンジのフォーラムで他の競技者によって詳しく書かれています。

In [27]:
kf=StratifiedKFold(n_splits=5, shuffle=True, random_state=2017)

やっと準備ができたところでレベル１モデル出力に進みましょう！

### 3.1 ランダムフォレスト
ランダムフォレストを使ってみましょう。

In [28]:
rf=RandomForestClassifier(n_estimators=200, n_jobs=6, min_samples_split=5, max_depth=7,
                          criterion='gini', random_state=0)

outcomes =cross_validate_sklearn(rf, x_train, y_train ,x_test, kf, scale=False, verbose=True)

rf_cv=outcomes[0]
rf_train_pred=outcomes[1]
rf_test_pred=outcomes[2]

rf_train_pred_df=pd.DataFrame(columns=['prediction_probability'], data=rf_train_pred)
rf_test_pred_df=pd.DataFrame(columns=['prediction_probability'], data=rf_test_pred)

fold cv 0 AUC score is 0.627738, Gini_Norm score is 0.255476
fold cv 1 AUC score is 0.628162, Gini_Norm score is 0.256324
fold cv 2 AUC score is 0.628498, Gini_Norm score is 0.256995
fold cv 3 AUC score is 0.624774, Gini_Norm score is 0.249548
fold cv 4 AUC score is 0.635280, Gini_Norm score is 0.270560
cv AUC score is 0.628841, Gini_Norm score is 0.257682
it takes 119.565 seconds to perform cross validation


### 3.2 決定木を増やしてみましょう
木をもっと足しちゃいましょう！

In [29]:
et=RandomForestClassifier(n_estimators=100, n_jobs=6, min_samples_split=5, max_depth=5,
                          criterion='gini', random_state=0)

outcomes =cross_validate_sklearn(et, x_train, y_train ,x_test, kf, scale=False, verbose=True)

et_cv=outcomes[0]
et_train_pred=outcomes[1]
et_test_pred=outcomes[2]

et_train_pred_df=pd.DataFrame(columns=['prediction_probability'], data=et_train_pred)
et_test_pred_df=pd.DataFrame(columns=['prediction_probability'], data=et_test_pred)

fold cv 0 AUC score is 0.623916, Gini_Norm score is 0.247832
fold cv 1 AUC score is 0.622989, Gini_Norm score is 0.245978
fold cv 2 AUC score is 0.624352, Gini_Norm score is 0.248703
fold cv 3 AUC score is 0.619732, Gini_Norm score is 0.239464
fold cv 4 AUC score is 0.630961, Gini_Norm score is 0.261922
cv AUC score is 0.624357, Gini_Norm score is 0.248714
it takes 47.035 seconds to perform cross validation


### 3.3 ロジスティック回帰
お馴染みのロジスティック回帰を使ってみましょう。

In [30]:
logit=LogisticRegression(random_state=0, C=0.5)

outcomes = cross_validate_sklearn(logit, x_train, y_train ,x_test, kf, scale=True, verbose=True)

logit_cv=outcomes[0]
logit_train_pred=outcomes[1]
logit_test_pred=outcomes[2]

logit_train_pred_df=pd.DataFrame(columns=['prediction_probability'], data=logit_train_pred)
logit_test_pred_df=pd.DataFrame(columns=['prediction_probability'], data=logit_test_pred)

fold cv 0 AUC score is 0.627116, Gini_Norm score is 0.254233
fold cv 1 AUC score is 0.624198, Gini_Norm score is 0.248397
fold cv 2 AUC score is 0.626947, Gini_Norm score is 0.253894
fold cv 3 AUC score is 0.626684, Gini_Norm score is 0.253367
fold cv 4 AUC score is 0.633531, Gini_Norm score is 0.267061
cv AUC score is 0.627682, Gini_Norm score is 0.255364
it takes 36.037 seconds to perform cross validation


### 3.4 ベルヌーイ分布
あまり馴染みのないナイーブベイズはXGBやLGBに匹敵するような出力を生成しないアルゴリズムではありますが、スタッキングの全体のパフォーマンスを向上させルために役立ちます。

In [31]:
nb=BernoulliNB()

outcomes =cross_validate_sklearn(nb, x_train, y_train ,x_test, kf, scale=True, verbose=True)

nb_cv=outcomes[0]
nb_train_pred=outcomes[1]
nb_test_pred=outcomes[2]

nb_train_pred_df=pd.DataFrame(columns=['prediction_probability'], data=nb_train_pred)
nb_test_pred_df=pd.DataFrame(columns=['prediction_probability'], data=nb_test_pred)

fold cv 0 AUC score is 0.618216, Gini_Norm score is 0.236431
fold cv 1 AUC score is 0.617390, Gini_Norm score is 0.234780
fold cv 2 AUC score is 0.619533, Gini_Norm score is 0.239066
fold cv 3 AUC score is 0.618452, Gini_Norm score is 0.236905
fold cv 4 AUC score is 0.628443, Gini_Norm score is 0.256886
cv AUC score is 0.620379, Gini_Norm score is 0.240758
it takes 29.043 seconds to perform cross validation


### 3.5 XGB
最強のGBMバズーカを使ってみましょう。

In [32]:
xgb_params = {
    "booster"  :  "gbtree", 
    "objective"         :  "binary:logistic",
    "tree_method": "hist",
    "eval_metric": "auc",
    "eta": 0.1,
    "max_depth": 5,
    "min_child_weight": 10,
    "gamma": 0.70,
    "subsample": 0.76,
    "colsample_bytree": 0.95,
    "nthread": 6,
    "seed": 0,
    'silent': 1
}

outcomes=cross_validate_xgb(xgb_params, x_train, y_train, x_test, kf, use_rank=False, verbose_eval=False)

xgb_cv=outcomes[0]
xgb_train_pred=outcomes[1]
xgb_test_pred=outcomes[2]

xgb_train_pred_df=pd.DataFrame(columns=['prediction_probability'], data=xgb_train_pred)
xgb_test_pred_df=pd.DataFrame(columns=['prediction_probability'], data=xgb_test_pred)


fold cv 0 AUC score is 0.642407, Gini_Norm score is 0.284814
fold cv 1 AUC score is 0.640669, Gini_Norm score is 0.281339
fold cv 2 AUC score is 0.642415, Gini_Norm score is 0.284829
fold cv 3 AUC score is 0.640377, Gini_Norm score is 0.280754
fold cv 4 AUC score is 0.645840, Gini_Norm score is 0.291680
cv AUC score is 0.642265, Gini_Norm score is 0.284530
it takes 121.876 seconds to perform cross validation


### 3.6 LightGBM

In [34]:
lgb_params = {
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 22,
    'min_sum_hessian_in_leaf': 20,
    'max_depth': 5,
    'learning_rate': 0.1,  # 0.618580
    'num_threads': 6,
    'feature_fraction': 0.6894,
    'bagging_fraction': 0.4218,
    'max_drop': 5,
    'drop_rate': 0.0123,
    'min_data_in_leaf': 10,
    'bagging_freq': 1,
    'lambda_l1': 1,
    'lambda_l2': 0.01,
    'verbose': 1
}


cat_cols=['ps_ind_02_cat','ps_car_04_cat', 'ps_car_09_cat','ps_ind_05_cat', 'ps_car_01_cat']
outcomes=cross_validate_lgb(lgb_params,x_train, y_train ,x_test,kf, cat_cols, use_cat=True, 
                            verbose_eval=False, use_rank=False)

lgb_cv=outcomes[0]
lgb_train_pred=outcomes[1]
lgb_test_pred=outcomes[2]

lgb_train_pred_df=pd.DataFrame(columns=['prediction_probability'], data=lgb_train_pred)
lgb_test_pred_df=pd.DataFrame(columns=['prediction_probability'], data=lgb_test_pred)

/home/yifan/.conda/envs/dragons/lib/python3.6/site-packages/lightgbm/basic.py:1030: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/yifan/.conda/envs/dragons/lib/python3.6/site-packages/lightgbm/basic.py:671: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


fold cv 0 AUC score is 0.640417, Gini_Norm score is 0.280834
fold cv 1 AUC score is 0.640079, Gini_Norm score is 0.280157
fold cv 2 AUC score is 0.641063, Gini_Norm score is 0.282127
fold cv 3 AUC score is 0.639120, Gini_Norm score is 0.278241
fold cv 4 AUC score is 0.646300, Gini_Norm score is 0.292600
cv AUC score is 0.640832, Gini_Norm score is 0.281665
it takes 65.087 seconds to perform cross validation


レベル１が準備できたとことで、次に進みましょう！

## 4. レベル２アンサンブル


### 4.1 レベル１出力データフレームの作成
レベル１のOOF予測結果をまとめてレベル２スタッキングの入力データを作りましょう。

In [35]:
columns=['rf','et','logit','nb','xgb','lgb']
train_pred_df_list=[rf_train_pred_df, et_train_pred_df, logit_train_pred_df, nb_train_pred_df,
                    xgb_train_pred_df, lgb_train_pred_df]

test_pred_df_list=[rf_test_pred_df, et_test_pred_df, logit_test_pred_df, nb_test_pred_df,
                    xgb_test_pred_df, lgb_test_pred_df]

lv1_train_df=pd.DataFrame(columns=columns)
lv1_test_df=pd.DataFrame(columns=columns)

for i in range(0,len(columns)):
    lv1_train_df[columns[i]]=train_pred_df_list[i]['prediction_probability']
    lv1_test_df[columns[i]]=test_pred_df_list[i]['prediction_probability']

### 4.2 レベル２ XGB
レベル２にもブースティングを使ってみましょう！

…やり方は先ほどと全く同じなのでしょうか？

In [36]:
xgb_lv2_outcomes=cross_validate_xgb(xgb_params, lv1_train_df, y_train, lv1_test_df, kf, 
                                          verbose=True, verbose_eval=False, use_rank=False)

xgb_lv2_cv=xgb_lv2_outcomes[0]
xgb_lv2_train_pred=xgb_lv2_outcomes[1]
xgb_lv2_test_pred=xgb_lv2_outcomes[2]

fold cv 0 AUC score is 0.642236, Gini_Norm score is 0.284472
fold cv 1 AUC score is 0.642141, Gini_Norm score is 0.284282
fold cv 2 AUC score is 0.643130, Gini_Norm score is 0.286260
fold cv 3 AUC score is 0.640631, Gini_Norm score is 0.281262
fold cv 4 AUC score is 0.647816, Gini_Norm score is 0.295632
cv AUC score is 0.581720, Gini_Norm score is 0.163440
it takes 33.358 seconds to perform cross validation


何かおかしいようです。CVスコアはなかなか良いですが、全体の学習CVスコアが落ちてしまいました。AUCとジニスコアはランキングにもとづいた指標なので、XGBとLGBをレベル２スタッキングで用いると、各フォルドで計算された予測スコアを合わせたときにランキングがおかしくなってしまうのです。実はこれを防ぐために先ほど確率をランクに変換する関数を書きました。

*use_rank*を使ってみましょう。

In [37]:
xgb_lv2_outcomes=cross_validate_xgb(xgb_params, lv1_train_df, y_train, lv1_test_df, kf, 
                                          verbose=True, verbose_eval=False, use_rank=True)

xgb_lv2_cv=xgb_lv2_outcomes[0]
xgb_lv2_train_pred=xgb_lv2_outcomes[1]
xgb_lv2_test_pred=xgb_lv2_outcomes[2]

fold cv 0 AUC score is 0.642236, Gini_Norm score is 0.284472
fold cv 1 AUC score is 0.642141, Gini_Norm score is 0.284282
fold cv 2 AUC score is 0.643130, Gini_Norm score is 0.286260
fold cv 3 AUC score is 0.640631, Gini_Norm score is 0.281262
fold cv 4 AUC score is 0.647816, Gini_Norm score is 0.295632
cv AUC score is 0.643200, Gini_Norm score is 0.286399
it takes 31.880 seconds to perform cross validation


学習予測のOOFスコアが上がり学習予測結果の良くなりました。レベル１で最もよかったのはXGBで得た0.282で今回は0.284が得られ、どのレベル１OOF学習スコアよりも良くなっていることがわかります。

### 4.3 レベル２ LightGBM
LightGBMも同様です。*use_rank*機能を使いましょう。

In [38]:
lgb_lv2_outcomes=cross_validate_lgb(lgb_params,lv1_train_df, y_train ,lv1_test_df,kf, [], use_cat=False, 
                                    verbose_eval=False, use_rank=True)

lgb_lv2_cv=xgb_lv2_outcomes[0]
lgb_lv2_train_pred=lgb_lv2_outcomes[1]
lgb_lv2_test_pred=lgb_lv2_outcomes[2]

fold cv 0 AUC score is 0.640239, Gini_Norm score is 0.280479
fold cv 1 AUC score is 0.639798, Gini_Norm score is 0.279596
fold cv 2 AUC score is 0.640237, Gini_Norm score is 0.280474
fold cv 3 AUC score is 0.638335, Gini_Norm score is 0.276670
fold cv 4 AUC score is 0.646149, Gini_Norm score is 0.292299
cv AUC score is 0.640982, Gini_Norm score is 0.281964
it takes 9.026 seconds to perform cross validation


### 4.3 レベル２ ランダムフォレスト
レベル２にもランダムフォレストを使ってみましょう。

In [39]:
rf_lv2=RandomForestClassifier(n_estimators=200, n_jobs=6, min_samples_split=5, max_depth=7,
                          criterion='gini', random_state=0)
rf_lv2_outcomes = cross_validate_sklearn(rf_lv2, lv1_train_df, y_train ,lv1_test_df, kf, 
                                            scale=True, verbose=True)
rf_lv2_cv=rf_lv2_outcomes[0]
rf_lv2_train_pred=rf_lv2_outcomes[1]
rf_lv2_test_pred=rf_lv2_outcomes[2]

fold cv 0 AUC score is 0.642115, Gini_Norm score is 0.284230
fold cv 1 AUC score is 0.640611, Gini_Norm score is 0.281222
fold cv 2 AUC score is 0.642669, Gini_Norm score is 0.285338
fold cv 3 AUC score is 0.640341, Gini_Norm score is 0.280683
fold cv 4 AUC score is 0.646928, Gini_Norm score is 0.293856
cv AUC score is 0.642027, Gini_Norm score is 0.284055
it takes 179.781 seconds to perform cross validation


### 4.4 レベル２ ロジスティック回帰

In [40]:
logit_lv2=LogisticRegression(random_state=0, C=0.5)
logit_lv2_outcomes = cross_validate_sklearn(logit_lv2, lv1_train_df, y_train ,lv1_test_df, kf, 
                                            scale=True, verbose=True)
logit_lv2_cv=logit_lv2_outcomes[0]
logit_lv2_train_pred=logit_lv2_outcomes[1]
logit_lv2_test_pred=logit_lv2_outcomes[2]

fold cv 0 AUC score is 0.641212, Gini_Norm score is 0.282425
fold cv 1 AUC score is 0.639367, Gini_Norm score is 0.278733
fold cv 2 AUC score is 0.641523, Gini_Norm score is 0.283046
fold cv 3 AUC score is 0.639232, Gini_Norm score is 0.278464
fold cv 4 AUC score is 0.646012, Gini_Norm score is 0.292024
cv AUC score is 0.641076, Gini_Norm score is 0.282152
it takes 6.127 seconds to perform cross validation


レベル１OOF出力で求めたメタフィーチャのおかげで、ランダムフォレストやロジスティック回帰といったモデルがレベル２で優れた結果を出すようになったことが確認できます。

ここでやめては面白くないのでレベル３もやってみましょう！

## 5. レベル３アンサンブル
レベル３でもレベル２に似た手順を追います。まずレベル２のOOF出力をまとめ、お好きな学習アルゴリズムに渡します。

### 5.1 レベル２出力データフレームの作成

In [41]:
lv2_columns=['rf_lf2', 'logit_lv2', 'xgb_lv2','lgb_lv2']
train_lv2_pred_list=[rf_lv2_train_pred, logit_lv2_train_pred, xgb_lv2_train_pred, lgb_lv2_train_pred]

test_lv2_pred_list=[rf_lv2_test_pred, logit_lv2_test_pred, xgb_lv2_test_pred, lgb_lv2_test_pred]

lv2_train=pd.DataFrame(columns=lv2_columns)
lv2_test=pd.DataFrame(columns=lv2_columns)

for i in range(0,len(lv2_columns)):
    lv2_train[lv2_columns[i]]=train_lv2_pred_list[i]
    lv2_test[lv2_columns[i]]=test_lv2_pred_list[i]

### 5.2 レベル３ XGB
ここではXGBのみ使ってみましょう。

In [42]:
xgb_lv3_params = {
    "booster"  :  "gbtree", 
    "objective"         :  "binary:logistic",
    "tree_method": "hist",
    "eval_metric": "auc",
    "eta": 0.1,
    "max_depth": 2,
    "min_child_weight": 10,
    "gamma": 0.70,
    "subsample": 0.76,
    "colsample_bytree": 0.95,
    "nthread": 6,
    "seed": 0,
    'silent': 1
}



xgb_lv3_outcomes=cross_validate_xgb(xgb_lv3_params, lv2_train, y_train, lv2_test, kf, 
                                          verbose=True, verbose_eval=False, use_rank=True)

xgb_lv3_cv=xgb_lv3_outcomes[0]
xgb_lv3_train_pred=xgb_lv3_outcomes[1]
xgb_lv3_test_pred=xgb_lv3_outcomes[2]

fold cv 0 AUC score is 0.642765, Gini_Norm score is 0.285529
fold cv 1 AUC score is 0.642432, Gini_Norm score is 0.284865
fold cv 2 AUC score is 0.642701, Gini_Norm score is 0.285402
fold cv 3 AUC score is 0.640837, Gini_Norm score is 0.281673
fold cv 4 AUC score is 0.647882, Gini_Norm score is 0.295764
cv AUC score is 0.643337, Gini_Norm score is 0.286674
it takes 44.188 seconds to perform cross validation


レベル２で求めたXBGの結果よりも少しだけ良いですが大差はなく、レベルを増やすことで収穫逓減が起こり始めてしまいました。線形的なものと組み合わせてみましょう。

### 5.3 レベル３ ロジスティック回帰
線形といえばロジスティック回帰ですね！

In [43]:
logit_lv3=LogisticRegression(random_state=0, C=0.5)
logit_lv3_outcomes = cross_validate_sklearn(logit_lv3, lv2_train, y_train ,lv2_test, kf, 
                                            scale=True, verbose=True)
logit_lv3_cv=logit_lv3_outcomes[0]
logit_lv3_train_pred=logit_lv3_outcomes[1]
logit_lv3_test_pred=logit_lv3_outcomes[2]


fold cv 0 AUC score is 0.642561, Gini_Norm score is 0.285122
fold cv 1 AUC score is 0.642091, Gini_Norm score is 0.284182
fold cv 2 AUC score is 0.643098, Gini_Norm score is 0.286195
fold cv 3 AUC score is 0.640703, Gini_Norm score is 0.281406
fold cv 4 AUC score is 0.647777, Gini_Norm score is 0.295554
cv AUC score is 0.643160, Gini_Norm score is 0.286320
it takes 4.816 seconds to perform cross validation


このレベルまでくるとXGBとロジスティック回帰の差はあまり見られません。

### 5.4 レベル３出力データの平均と回答の提出
２つのウェイトの平均をとり、もう少し絞れないか見てみましょう。

In [44]:
weight_avg=logit_lv3_train_pred*0.5+ xgb_lv3_train_pred*0.5
print(auc_to_gini_norm(roc_auc_score(y_train, weight_avg)))

0.286698636514


学習スコアは0.28443と求まりました。回答の提出をするためテストデータの方も同じウェイトで掛けましょう。

In [45]:
submission=sample_submission.copy()
submission['target']=logit_lv3_test_pred*0.5+ xgb_lv3_test_pred*0.5
filename='stacking_demonstration.csv.gz'
submission.to_csv(filename,compression='gzip', index=False)

## 6 終えてみて
レベルを３つ使ったスタッキングのやり方をご理解いただけたでしょうか。学習データからより多くの情報を取り出すことで、より良いテスト予測が期待できます。しかし実際このチャレンジのデータにはノイズが多く、レベル２以上のスタッキングがスコアを向上させてくれる保証は正直ないような気もします。

以下の手順でスタッキングを進めるのも案です。

レベル２まで進めウェイトを平均する
いくつか異なった乱数シードに対しても同じような手順でスタッキングを行う
全ての平均をとる

CV（交差検証）もこのチャレンジの鍵になるかと思います。もしかしたら締め切り前日になって誰かが0.291のスクリプトを流出させて私たちをこう興奮させてくれるかもしれません。締め切りわずかの時間を楽しんでくださいね！被保険者が請求する必要がないことを願って。